# Homework: Competitive Grammar Writing

In [2]:
from pcfg_parse_gen import Pcfg, PcfgGenerator, CkyParse
import nltk

def print_tree(tree_string):
    tree_string = tree_string.strip()
    tree = nltk.Tree.fromstring(tree_string)
    tree.pretty_print()

def draw_tree(tree_string):
    tree_string = tree_string.strip()
    tree = nltk.Tree.fromstring(tree_string)
    tree.draw()

## Generate S1.gr

### Produce parsed trees from the Penn Treebank

In [ ]:
import benepar

bst_parser = benepar.Parser("benepar_en")
with open('example_sentences.txt') as fh:
    sents = [line.strip() for line in fh]

trees = [str(bst_parser.parse(sent)) for sent in sents]
with open('example_sentences.trees', 'w') as f:
    for tree in trees:
        f.write(tree)
        f.write('\n')

### Generate rules from parsed trees

- Read parsed trees with BracketParseCorpusReader.
- Convert each tree to its chomsky normal form.
- Generate the productions (rules) that correspond to the non-terminal nodes of the tree.
- Count and save number of occurrences for each rule as weight.
- Count and save the number of occurrences for each type of sentences, e.g., S, SQ, SBARQ.
- Write the rules and manually modify the rules that generate cycle erros.
- Adjust weights for S and S2 accordingly.

In [15]:
from nltk.corpus.reader import BracketParseCorpusReader
from nltk import treetransforms

def train(training):
    """
    Generate maximum likelihood estimates of
    productions from a training file of s-expression trees.
    """
    reader = BracketParseCorpusReader('.', training)
    counts = {}
    labels = {}
    for tree in reader.parsed_sents():
        tree.chomsky_normal_form(horzMarkov=2)
        label = tree.label()
        if label not in labels:
            labels[label] = 0
        labels[label] += 1
        productions = tree.productions()
        for production in productions:
            if production not in counts:
                counts[production] = 0
            counts[production] += 1
        # productions += tree.productions()

    lhs_map = {}
    for production in counts.keys():
        if production.lhs() not in lhs_map:
            lhs_map[production.lhs()] = []
        lhs_map[production.lhs()].append(production)

    keys = lhs_map.keys()
    keys = sorted(keys)

    weights = {}
    for _, productions in lhs_map.items():
        for production in productions:
            weights[production] = counts[production]
    return weights, labels, lhs_map

def write_rules(lhs_map, labels, weights, allowed_words, grammar):
    with open(allowed_words, 'r') as f:
        lines = f.readlines()
    allowed_words = set(map(lambda l: l.strip(), lines))
    
    # print(lhs_map)
    keys = lhs_map.keys()
    keys = sorted(keys)
    with open(grammar, 'w') as f:
        # f.write('99\tTOP\tS\n')
        f.write('10\tTOP\tS2\n')
        for label, count in labels.items():
            f.write("%d\tTOP\t%s\n" % (count, label))
        for key in keys:
            productions = lhs_map[key]
            # f.write("#\n")
            for production in productions:
                valid_mask = (len(str(production.lhs())) > 0)
                rhs_symbols = [str(nt) for nt in production.rhs()]
#                 print(rhs_symbols)
                vocab_mask = (len(rhs_symbols) == 1) & (rhs_symbols[0] in allowed_words)
                if valid_mask & (not vocab_mask):                    
                    f.write("%d\t%s\t%s\n" % (weights[production],
                                              production.lhs(),
                                              ' '.join(rhs_symbols)))

Train grammar from example_sentences.trees and/or devset.trees

In [16]:
training = 'example_sentences.trees' # only for example_sentences.trees
# training = 'training.trees'  # for example_sentences.trees and devset.trees
grammar = 'S2_tuned.gr'
allowed_words = 'allowed_words.txt'

weights, labels, lhs_map = train(training)
write_rules(lhs_map, labels, weights, allowed_words, grammar)

### Parsing sentences with trained grammar and comparing with the default grammar

In [134]:
parse_gram = Pcfg(["S1.gr","S2.gr", "Vocab.gr"])
parse_gram_tuned = Pcfg(["S1_tunedS.gr", "S2_yabing.gr", 'Vocab_yabing.gr'])

#reading grammar file: S1.gr
#reading grammar file: S2.gr
#reading grammar file: Vocab.gr
#reading grammar file: S1_tunedS.gr
#reading grammar file: S2_yabing.gr
#reading grammar file: Vocab_yabing.gr


In [136]:
parser = CkyParse(parse_gram, beamsize=0.00001)
parser_tuned = CkyParse(parse_gram_tuned, beamsize=0.00001)
ce, trees = parser.parse_sentences(["Arthur is the king ."])
print("-cross entropy: {}".format(ce))
for tree_string in trees:
    print_tree(tree_string)

(TOP (S1 (NP (Proper Arthur) ) (_VP (VP (VerbT is) (NP (Det the) (Nbar (Noun king) ))) (Punc .))) )
-cross entropy: -3.7863679540999504
             TOP              
              |                
              S1              
   ___________|___             
  |              _VP          
  |            ___|________    
  |           VP           |  
  |       ____|___         |   
  |      |        NP       |  
  |      |     ___|___     |   
  NP     |    |      Nbar  |  
  |      |    |       |    |   
Proper VerbT Det     Noun Punc
  |      |    |       |    |   
Arthur   is  the     king  .  



#parsing: ['Arthur', 'is', 'the', 'king', '.']
#-cross entropy (bits/word): -3.78637


In [142]:
ce, trees = parser_tuned.parse_sentences(["Arthur is a good king ."])
print("-cross entropy: {}".format(ce))
for tree_string in trees:
    print_tree(tree_string)

(TOP (S (NP (NNP Arthur) ) (S|<VP-END> (VP (VBZ is) (NP (DT a) (NP|<JJ-NN> (JJ good) (NN king)))) (END .))) )
-cross entropy: -6.760388705012244
           TOP                                   
            |                                     
            S                                    
   _________|__________                           
  |                S|<VP-END>                    
  |              ______|_______________________   
  |             VP                             | 
  |      _______|______                        |  
  |     |              NP                      | 
  |     |    __________|__________             |  
  NP    |   |                 NP|<JJ-NN>       | 
  |     |   |           __________|_______     |  
 NNP   VBZ  DT         JJ                 NN  END
  |     |   |          |                  |    |  
Arthur  is  a         good               king  . 



#parsing: ['Arthur', 'is', 'a', 'good', 'king', '.']
#-cross entropy (bits/word): -6.76039


In [138]:
ce, trees = parser.parse_sentences(["either Arthur knows or Patsy does ."])
print("-cross entropy: {}".format(ce))
for tree_string in trees:
    print_tree(tree_string)

(TOP (S2 (_Misc (Misc either) (_Proper (Proper Arthur) (_Misc (Misc knows) (_Misc (Misc or) (_Proper (Proper Patsy) (_Misc (Misc does) (_Misc (Misc .) ))))))) ) )
-cross entropy: -11.11731610342174
        TOP                                                 
         |                                                   
         S2                                                 
         |                                                   
       _Misc                                                
   ______|_____________                                      
  |                 _Proper                                 
  |       _____________|______                               
  |      |                  _Misc                           
  |      |       _____________|______                        
  |      |      |                  _Misc                    
  |      |      |       _____________|______                 
  |      |      |      |                 _Proper            
  |

#parsing: ['either', 'Arthur', 'knows', 'or', 'Patsy', 'does', '.']
#-cross entropy (bits/word): -11.1173


In [139]:
ce, trees = parser_tuned.parse_sentences(["either Arthur knows or Patsy does ."])
print("-cross entropy: {}".format(ce))
for tree_string in trees:
    print_tree(tree_string)

(TOP (S (S (NP (DT either) (NNP Arthur)) (VP (VBZ knows) )) (S|<CC-S> (CC or) (S|<S-END> (S (NP (NNP Patsy) ) (VP (VBZ does) )) (END .)))) )
-cross entropy: -9.553873810123278
                        TOP                             
                         |                               
                         S                              
              ___________|___________                    
             |                    S|<CC-S>              
             |            ___________|_________          
             |           |                 S|<S-END>    
             |           |            _________|______   
             S           |           S                | 
         ____|______     |     ______|_________       |  
        NP          VP   |    NP               VP     | 
   _____|____       |    |    |                |      |  
  DT        NNP    VBZ   CC  NNP              VBZ    END
  |          |      |    |    |                |      |  
either     Arthur k

#parsing: ['either', 'Arthur', 'knows', 'or', 'Patsy', 'does', '.']
#-cross entropy (bits/word): -9.55387


Use `parse_file` to parse a file of sentences.

In [141]:
ce, trees = parser_tuned.parse_file('devset.txt')
print("-cross entropy: {}".format(ce))

#parsing: ['Whoa', 'there', '!']
#parsing: ['Halt', '!']
#parsing: ['Who', 'goes', 'there', '?']
#parsing: ['It', 'is', 'I', ',', 'Arthur', ',', 'son', 'of', 'Uther', 'Pendragon', ',', 'from', 'the', 'castle', 'of', 'Camelot', '.']


(TOP (INTJ (NNP Whoa) (INTJ|<RB-END> (RB there) (END !))) )
(TOP (S (VP (VB Halt) ) (END !)) )
(TOP (SBARQ (WHNP (WP Who) ) (SBARQ|<SQ-END> (SQ (VBZ goes) (NP (EX there) )) (END ?))) )
(TOP (S2 (_PRP (PRP It) (_VBZ (VBZ is) (_PRP (PRP I) (_PAUSE (PAUSE ,) (_NNP (NNP Arthur) (_PAUSE (PAUSE ,) (_NN (NN son) (_IN (IN of) (_NNP (NNP Uther) (_NNP (NNP Pendragon) (_PAUSE (PAUSE ,) (_IN (IN from) (_DT (DT the) (_NN (NN castle) (_IN (IN of) (_NNP (NNP Camelot) (_END (END .) ))))))))))))))))) ) )


#parsing: ['King', 'of', 'the', 'Britons', ',', 'defeator', 'of', 'the', 'Saxons', ',', 'sovereign', 'of', 'all', 'England', '!']


(TOP (S2 (_NNP (NNP King) (_IN (IN of) (_DT (DT the) (_NNPS (NNPS Britons) (_PAUSE (PAUSE ,) (_NN (NN defeator) (_IN (IN of) (_DT (DT the) (_NNS (NNS Saxons) (_PAUSE (PAUSE ,) (_JJ (JJ sovereign) (_IN (IN of) (_DT (DT all) (_NNP (NNP England) (_END (END !) ))))))))))))))) ) )
(TOP (S (VP (VB Pull) (NP (DT the) (NP|<JJ-CD> (JJ other) (CD one)))) (END !)) )
(TOP (S (NP (PRP I) ) (S|<VP-END> (VP (VBP am) ) (END .))) )
(TOP (FRAG (CC And) (FRAG|<NP-NP> (NP (DT this) (NP (PRP$ my) (NP|<JJ-NN> (JJ trusty) (NN servant)))) (FRAG|<NP-END> (NP (NNP Patsy) ) (END .)))) )


#parsing: ['Pull', 'the', 'other', 'one', '!']
#parsing: ['I', 'am', '.']
#parsing: ['And', 'this', 'my', 'trusty', 'servant', 'Patsy', '.']
#parsing: ['We', 'have', 'ridden', 'the', 'length', 'and', 'breadth', 'of', 'the', 'land', 'in', 'search', 'of', 'knights', 'who', 'will', 'join', 'me', 'in', 'my', 'court', 'of', 'Camelot', '.']


...

#parsing: ['Well', 'its', 'only', 'because', 'you', 'do', "n't", 'brush', 'my', 'teeth', '.']


(TOP (S2 (_RB (RB Well) (_PRP$ (PRP$ its) (_RB (RB only) (_IN (IN because) (_PRP (PRP you) (_VBP (VBP do) (_RB (RB n't) (_NN (NN brush) (_PRP$ (PRP$ my) (_NNS (NNS teeth) (_END (END .) ))))))))))) ) )
(TOP (S2 (_UH (UH Oh) (_VB (VB stop) (_VBG (VBG bitching) (_CC (CC and) (_VB (VB let) (_POS (POS 's) (_VBP (VBP go) (_VBP (VBP have) (_NN (NN tea) (_END (END .) )))))))))) ) )


#parsing: ['Oh', 'stop', 'bitching', 'and', 'let', "'s", 'go', 'have', 'tea', '.']
#parsing: ['All', 'right', ',', 'all', 'right', ',', 'all', 'right', '.']


(TOP (S2 (_PDT (PDT All) (_RB (RB right) (_PAUSE (PAUSE ,) (_PDT (PDT all) (_RB (RB right) (_PAUSE (PAUSE ,) (_PDT (PDT all) (_RB (RB right) (_END (END .) ))))))))) ) )
(TOP (S (NP (PRP We) ) (S|<VP-END> (VP (VP (MD 'll) (VP (VB kill) (VP|<NP-ADVP> (NP (PRP him) ) (ADVP (RB first) )))) (VP|<CC-RB> (CC and) (VP|<RB-VP> (RB then) (VP (VBP have) (NP (NN tea) (NP|<CC-NNS> (CC and) (NNS biscuits))))))) (END .))) )
(TOP (S2 (_UH (UH Yes) (_END (END .) )) ) )
(TOP (FRAG (INTJ (UH Oh) ) (FRAG|<PAUSE-CC> (PAUSE ,) (FRAG|<CC-RB> (CC but) (FRAG|<RB-NP> (RB not) (FRAG|<NP-END> (NP (NNS biscuits) ) (END .)))))) )
-cross entropy: -8.416141890351343


#parsing: ['We', "'ll", 'kill', 'him', 'first', 'and', 'then', 'have', 'tea', 'and', 'biscuits', '.']
#parsing: ['Yes', '.']
#parsing: ['Oh', ',', 'but', 'not', 'biscuits', '.']
#-cross entropy (bits/word): -8.41614


In [143]:
ce, trees = parser_tuned.parse_file('example_sentences.txt')
print("-cross entropy: {}".format(ce))

#parsing: ['Arthur', 'is', 'the', 'king', '.']
#parsing: ['Arthur', 'rides', 'the', 'horse', 'near', 'the', 'castle', '.']
#parsing: ['riding', 'to', 'Camelot', 'is', 'hard', '.']
#parsing: ['do', 'coconuts', 'speak', '?']
#parsing: ['what', 'does', 'Arthur', 'ride', '?']
#parsing: ['who', 'does', 'Arthur', 'suggest', 'she', 'carry', '?']


(TOP (S (NP (NNP Arthur) ) (S|<VP-END> (VP (VBZ is) (NP (DT the) (NN king))) (END .))) )
(TOP (S2 (_NNP (NNP Arthur) (_NNS (NNS rides) (_DT (DT the) (_NN (NN horse) (_IN (IN near) (_DT (DT the) (_NN (NN castle) (_END (END .) )))))))) ) )
(TOP (S2 (_VBG (VBG riding) (_TO (TO to) (_NNP (NNP Camelot) (_VBZ (VBZ is) (_RB (RB hard) (_END (END .) )))))) ) )
(TOP (SQ (VBP do) (SQ|<NP-VP> (NP (NNS coconuts) ) (SQ|<VP-END> (VP (VB speak) ) (END ?)))) )
(TOP (SBARQ (WHNP (WP what) ) (SBARQ|<SQ-END> (SQ (VBZ does) (SQ|<NP-VP> (NP (NNP Arthur) ) (VP (VB ride) ))) (END ?))) )
(TOP (SBARQ (WHNP (WP who) ) (SBARQ|<SQ-END> (SQ (VBZ does) (SQ|<NP-SQ> (NP (NNP Arthur) ) (SQ (VB suggest) (SQ|<NP-VP> (NP (PRP she) ) (VP (VB carry) ))))) (END ?))) )
(TOP (SBARQ (WHADVP (WRB why) ) (SBARQ|<SQ-END> (SQ (VBZ does) (SQ|<NP-VP> (NP (NNP England) ) (VP (VBP have) (NP (DT a) (NN king))))) (END ?))) )


#parsing: ['why', 'does', 'England', 'have', 'a', 'king', '?']
#parsing: ['are', 'they', 'suggesting', 'Arthur', 'ride', 'to', 'Camelot', '?']
#parsing: ['five', 'strangers', 'are', 'at', 'the', 'Round', 'Table', '.']
#parsing: ['Guinevere', 'might', 'have', 'known', '.']
#parsing: ['Guinevere', 'should', 'be', 'riding', 'with', 'Patsy', '.']


(TOP (SQ (VBP are) (SQ|<NP-VP> (NP (PRP they) ) (VP (VBG suggesting) (S (NP (NNP Arthur) ) (S|<VP-END> (VP (VB ride) (PP (TO to) (NP (NNP Camelot) ))) (END ?)))))) )
(TOP (S (NP (CD five) (NNS strangers)) (S|<VP-END> (VP (VBP are) (PP (IN at) (NP (DT the) (NP|<NNP-NNP> (NNP Round) (NNP Table))))) (END .))) )
(TOP (S2 (_NNP (NNP Guinevere) (_MD (MD might) (_VBP (VBP have) (_VBN (VBN known) (_END (END .) ))))) ) )
(TOP (S (NP (NNP Guinevere) ) (S|<VP-END> (VP (MD should) (VP (VB be) (VP (VBG riding) (PP (IN with) (NP (NNP Patsy) ))))) (END .))) )
(TOP (S2 (_PRP (PRP it) (_VBZ (VBZ is) (_NNP (NNP Sir) (_NNP (NNP Lancelot) (_WP (WP who) (_VBZ (VBZ knows) (_NNP (NNP Zoot) (_END (END !) )))))))) ) )


#parsing: ['it', 'is', 'Sir', 'Lancelot', 'who', 'knows', 'Zoot', '!']
#parsing: ['either', 'Arthur', 'knows', 'or', 'Patsy', 'does', '.']
#parsing: ['neither', 'Sir', 'Lancelot', 'nor', 'Guinevere', 'will', 'speak', 'of', 'it', '.']


...

#parsing: ['Arthur', 'and', 'Guinevere', 'migrate', 'frequently', '.']
#parsing: ['he', 'knows', 'what', 'they', 'are', 'covering', 'with', 'that', 'story', '.']
#parsing: ['Arthur', 'suggested', 'that', 'the', 'castle', 'be', 'carried', '.']


(TOP (S (NP (PRP he) ) (S|<VP-END> (VP (VBZ knows) (SBAR (WHNP (WP what) ) (S (NP (PRP they) ) (VP (VBP are) (VP (VBG covering) (PP (IN with) (NP (DT that) (NN story)))))))) (END .))) )
(TOP (S2 (_NNP (NNP Arthur) (_VBD (VBD suggested) (_IN (IN that) (_DT (DT the) (_NN (NN castle) (_VB (VB be) (_VBN (VBN carried) (_END (END .) )))))))) ) )


#parsing: ['the', 'king', 'drank', 'to', 'the', 'castle', 'that', 'was', 'his', 'home', '.']


(TOP (S (NP (DT the) (NN king)) (S|<VP-END> (VP (VBD drank) (VP|<PP-PP> (PP (TO to) (NP (DT the) (NN castle))) (PP (IN that) (S (VP (VBD was) (NP (PRP$ his) (NN home))) )))) (END .))) )
(TOP (SBARQ (SBAR (WHADVP (WRB when) ) (S (NP (DT the) (NN king)) (VP (NNS drinks) ))) (SBARQ|<PAUSE-NP> (PAUSE ,) (SBARQ|<NP-END> (NP (NNP Patsy) (NNS drinks)) (END .)))) )
-cross entropy: -8.338544384278855


#parsing: ['when', 'the', 'king', 'drinks', ',', 'Patsy', 'drinks', '.']
#-cross entropy (bits/word): -8.33854


### Generating sentences with trained grammar

In [17]:
gen_gram = Pcfg(["S1_yabing.gr","Vocab_yabing.gr"])

#reading grammar file: S1_yabing.gr
#reading grammar file: Vocab_yabing.gr


In [18]:
gen = PcfgGenerator(gen_gram)
for _ in range(50):
    print(" ".join(gen.generate()))

he lying for Arthur
kaniggets Round , yours year , !
snakes might understand We're 's . !
What 's King brought being to Tim , Prince .
you 's you .
haw , the The set But Y'know , you , , coming in away .
the Grail fart to sixteen people .
haw guard !
the Tale profane leads you ?
English Listen Yapping out the nose !
Sir find
mumble auntie it
Haw thank .
This Sir haw is social .
Who are my spam used nee to that end !
tell leads
come chickening to Concorde
You 're to the sir
your room grenade Look is us , Bridge
fifty arrows praised from Thee Launcelot ?
No seek the pointy color and pestilence , lie kaniggets , answer swallow looking to a flight , y' , . .
the oral minute threw than Sir nee towards Knights ! .
a afraid sir was who Swamp !
my idea imprisoned !
is the Sir Concorde carried Round ?
Arthur be seemed sworn Packing against the father
he keep if Robin and Y'know carved ?
We're sign to You ?
what 's Guinevere 's Run . .
the hand be change legally carries and saved to you ?
Where 